In [1]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

C:\Users\alimu\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
text_df = pd.read_csv("fake_or_real_news.csv")
text = list(text_df.text.values)
joined_text = " ".join(text)

with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [3]:
partial_text = joined_text[:100000] # in case of more a powerful hardware increase the number of joined characters for better predictions

In [4]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [5]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [6]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [7]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [8]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

C:\Users\alimu\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 16s 88ms/step - accuracy: 0.0472 - loss: 7.1313
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 11s 85ms/step - accuracy: 0.0596 - loss: 6.6919
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 13s 94ms/step - accuracy: 0.0654 - loss: 6.5317
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 14s 100ms/step - accuracy: 0.0741 - loss: 6.3837
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - accuracy: 0.0874 - loss: 6.1860
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 13s 95ms/step - accuracy: 0.1023 - loss: 5.9198
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 13s 93ms/step - accuracy: 0.1122 - loss: 5.6487
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - accuracy: 0.1356 - loss: 5.3729
Epoch 9/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 14s 101ms/step - accuracy: 0.1570 - loss: 5.1308
Epoch 10/10
 44/135 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - accuracy: 0.1953 - loss: 4.7088

In [ ]:
model.save('textgenmodel_two.keras')

In [ ]:
model = load_model("textgenmodel_two.keras")

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
possible = predict_next_word("Hello my name is ", 5)

In [ ]:
possible # gives us the index number of the possible next words

In [ ]:
for idx in possible:
    print(unique_tokens[idx])

In [ ]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [ ]:
generate_text("My friend John is ", 10, 2)